In [1]:
import numpy as np
import pandas as pd

from desdeo_problem.Objective import _ScalarObjective
from desdeo_problem.Variable import variable_builder
from desdeo_problem.Problem import MOProblem
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_emo.othertools.plotlyanimate import animate_init_, animate_next_
from pygmo import fast_non_dominated_sorting as nds
import plotly.express as ex
from desdeo_problem.Constraint import ScalarConstraint

## MOP-C4 Tanaka from Coello's book

The problem is wrong in the book, look up literature.

### Define objectives and constraints

Note:

* Objectives should be defined as functions that take decision variables in the form of 2-d numpy arrays as inputs. Each row should represent a single individual. Each column should represent a single decision variable.
* Constraints should be defined as functions that take two numpy arrays as their inputs. The first array represent the decision variables, the second represents the objective function values. Hence, both, the decision variable, and the corresponding objective values can be used to form constraints.
* A constraint value of less than or equal to zero is considered feasible.

In [2]:
def f_1(x):
    return x[:,0]

def f_2(x):
    return x[:,1]

def c_1(x, y):
    a = 0.1
    b = 16
    #a = 0.1 * (x[:,0]**2 + x[:,1]**2 + 5 * x[:,0]*x[:,1])
    #b = 8 * (x[:,0]**2 + x[:,1]**2)
    return -x[:,0]**2 - x[:,1]**2 + 1 + a* np.cos(b*np.arctan(x[:,0]/x[:,1]))

def c_2(x,y):
    return - 0.5 + (x[:,0]-0.5)**2 + (x[:,1]-0.5)**2

### Build variables

See the repository desdeo_problem for more details about creating variable, objective, and problem objects.

In [3]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[0, 0],
                             upper_bounds=[np.pi, np.pi])

### Build objectives

In [4]:
f1 = _ScalarObjective(name='f1', evaluator=f_1)
f2 = _ScalarObjective(name='f2', evaluator=f_2)

### Build constraints

In [5]:
c1 = ScalarConstraint("c1", 2, 2, c_1)
c2 = ScalarConstraint("c2", 2, 2, c_2)

### Build the problem object

In [6]:
problem = MOProblem(variables=list_vars, objectives=[f1,f2], constraints=[c1,c2])

In [7]:
evolver = RVEA(problem, lattice_resolution=200, n_gen_per_iter=50, n_iterations=10)

In [8]:
while evolver.continue_evolution():
    evolver.iterate()

/home/rishi/.virtualenvs/desdeo-emo/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

divide by zero encountered in true_divide

/home/rishi/.virtualenvs/desdeo-emo/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in true_divide



In [9]:
objectives = evolver.population.objectives
non_dom = objectives[nds(objectives)[0][0]]

In [10]:
ex.scatter(x=non_dom[:,0], y=non_dom[:,1])

## Viennet 4

In [11]:
def f_1(x):
    return ((x[:,0] - 2) ** 2) / 2 + ((x[:,1] + 1) ** 2) / 13 + 3

def f_2(x):
    return ((x[:,0] + x[:,1] - 3)**2)/175+ ((2*x[:,1]-x[:,0])**2)/17 -13

def f_3(x):
    return ((3*x[:,0]-2*x[:,1]+4)**2)/8 + ((x[:,0]-x[:,1]+1)**2)/27 + 15

In [12]:
def c_1(x,y):
    return -3*x[:,0]+4-x[:,1] - np.finfo(float).eps

def c_2(x,y):
    return x[:,0] + 1 - np.finfo(float).eps

def c_3(x,y):
    return x[:,1]-x[:,0]+ 2-np.finfo(float).eps

In [13]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-4, -4],
                             upper_bounds=[4, 4])

In [14]:
f1 = _ScalarObjective(name='f1', evaluator=f_1)
f2 = _ScalarObjective(name='f2', evaluator=f_2)
f3 = _ScalarObjective(name='f3', evaluator=f_3)

In [15]:
c1 = ScalarConstraint("c1", n_decision_vars=2, n_objective_funs=3, evaluator=c_1)
c2 = ScalarConstraint("c2", n_decision_vars=2, n_objective_funs=3, evaluator=c_2)
c3 = ScalarConstraint("c3", n_decision_vars=2, n_objective_funs=3, evaluator=c_3)


In [16]:
problem = MOProblem(variables=list_vars, objectives=[f1,f2, f3], constraints=[c1,c2,c3])

In [17]:
evolver = RVEA(problem, lattice_resolution=25)

In [18]:
figure = animate_init_(evolver.population.objectives, filename="MOPC4.html")
while evolver.continue_evolution():
    evolver.iterate()
    figure = animate_next_(
        evolver.population.objectives,
        figure,
        filename="MOPC4.html",
        generation=evolver._iteration_counter,
    )

Plot saved as:  MOPC4.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


In [19]:
non_dom = evolver.population.objectives[nds(evolver.population.objectives)[0][0]]
non_dom = pd.DataFrame(non_dom, columns=["f1","f2", "f3"])

In [20]:
ex.scatter_3d(non_dom, x="f1", y="f2", z="f3", size=[1]*non_dom.shape[0], size_max=15)